# Using Ragas to Evaluate a RAG Application built with LangChain and LangGraph

In the following notebook, we'll be looking at how [Ragas](https://github.com/explodinggradients/ragas) can be helpful in a number of ways when looking to evaluate your RAG applications!

While this example is rooted in LangChain/LangGraph - Ragas is framework agnostic (you don't even need to be using a framework!).

- 🤝 Breakout Room #1
  1. Task 1: Installing Required Libraries
  2. Task 2: Set Environment Variables
  3. Task 3: Synthetic Dataset Generation for Evaluation using Ragas (Optional)

- 🤝 Breakout Room #2
  1. Task 4: Evaluating our Pipeline with Ragas
  2. Task 5: Testing OpenAI's Claim
  3. Task 6: Selecting an Advanced Retriever and Evaluating

But first! Let's set some dependencies!

## Dependencies and API Keys:

In [1]:
!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/5

In [2]:
!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.6/306.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.8/166.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━

We'll also need to provide our API keys.

First, OpenAI's for our LLM/embedding model combination!

In [3]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key!")

Please enter your OpenAI API key!··········


OPTIONALLY:

We can also provide a Ragas API key - which you can sign-up for [here](https://app.ragas.io/).

In [4]:
os.environ["RAGAS_APP_TOKEN"] = getpass("Please enter your Ragas API key!")

Please enter your Ragas API key!··········


## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [5]:
!mkdir data

In [6]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31287    0 31287    0     0  34833      0 --:--:-- --:--:-- --:--:-- 34802


In [7]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70146    0 70146    0     0  74935      0 --:--:-- --:--:-- --:--:-- 74862


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [8]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

In [10]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [11]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [12]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Wht did we lern about LLMs in 2024?,[Code may be the best application The ethics o...,The most surprising thing we’ve learned about ...,single_hop_specifc_query_synthesizer
1,How do LLMs handle the complexity of Chinese g...,[Based Development As a computer scientist and...,The grammar rules of programming languages lik...,single_hop_specifc_query_synthesizer
2,What were the key developments in AI in 2023?,[Simon Willison’s Weblog Subscribe Stuff we fi...,2023 was the breakthrough year for Large Langu...,single_hop_specifc_query_synthesizer
3,What plausible impact AI models have on jobs?,[easy to follow. The rest of the document incl...,People have certainly lost work to AI models—a...,single_hop_specifc_query_synthesizer
4,How have advancements in model efficiency and ...,[<1-hop>\n\nVoice and live camera mode are sci...,Advancements in model efficiency and pricing h...,multi_hop_abstract_query_synthesizer
5,How has the universal access to AI models evol...,[<1-hop>\n\nVoice and live camera mode are sci...,Universal access to the best AI models was bri...,multi_hop_abstract_query_synthesizer
6,How have advancements in model efficiency and ...,[<1-hop>\n\nVoice and live camera mode are sci...,Advancements in model efficiency and pricing h...,multi_hop_abstract_query_synthesizer
7,How has universal access to AI models and API ...,[<1-hop>\n\nVoice and live camera mode are sci...,Universal access to AI models experienced a br...,multi_hop_abstract_query_synthesizer
8,What were the significant advancements in Larg...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,"In 2024, significant advancements in Large Lan...",multi_hop_specific_query_synthesizer
9,How has the development of Claude 3 and the ri...,[<1-hop>\n\nI’m beginning to see the most popu...,The development of Claude 3 and the rise of in...,multi_hop_specific_query_synthesizer


#### OPTIONAL:

If you've provided your Ragas API key - you can use this web interface to look at the created data!

In [13]:
dataset.upload()

Testset uploaded! View at https://app.ragas.io/dashboard/alignment/testset/a2c7f697-fd3a-4abb-9643-d8f081e474b5


'https://app.ragas.io/dashboard/alignment/testset/a2c7f697-fd3a-4abb-9643-d8f081e474b5'

## LangChain RAG

Now we'll construct our LangChain RAG, which we will be evaluating using the above created test data!

### R - Retrieval

Let's start with building our retrieval pipeline, which will involve loading the same data we used to create our synthetic test set above.

> NOTE: We need to use the same data - as our test set is specifically designed for this data.

In [14]:
path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

Now that we have our data loaded, let's split it into chunks!

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

74

Next up, we'll need to provide an embedding model that we can use to construct our vector store.

In [16]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Now we can build our in memory QDrant vector store.

In [17]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="ai_across_years",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="ai_across_years",
    embedding=embeddings,
)

We can now add our documents to our vector store.

In [18]:
_ = vector_store.add_documents(documents=split_documents)

Let's define our retriever.

In [19]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

Now we can produce a node for retrieval!

In [20]:
def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

### Augmented

Let's create a simple RAG prompt!

In [21]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### Generation

We'll also need an LLM to generate responses - we'll use `gpt-4o-mini` to avoid using the same model as our judge model.

In [22]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Then we can create a `generate` node!

In [23]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

### Building RAG Graph with LangGraph

Let's create some state for our LangGraph RAG graph!

In [24]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

Now we can build our simple graph!

> NOTE: We're using `add_sequence` since we will always move from retrieval to generation. This is essentially building a chain in LangGraph.

In [25]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

Let's do a test to make sure it's doing what we'd expect.

In [26]:
response = graph.invoke({"question" : "How are LLM agents useful?"})

In [27]:
response["response"]

"LLM agents are useful for several reasons:\n\n1. **Ease of Building**: Creating LLMs has become relatively straightforward, requiring only a few hundred lines of Python code, as opposed to millions of lines of complex code that one might expect for such powerful systems. This accessibility means that more individuals and organizations can experiment with and deploy LLMs.\n\n2. **Data-Driven Performance**: The effectiveness of LLMs largely depends on the quality and quantity of the training data. If one can gather the right data and afford the necessary computational resources, building an effective LLM is achievable.\n\n3. **Running Locally**: Recent advancements, such as the release of models like Meta's Llama, have enabled users to run LLMs on their own devices. This democratizes access to LLM technology, allowing individuals to utilize these models without needing expensive servers.\n\n4. **Code Generation**: LLMs have proven particularly effective at generating code. They can prod

## Evaluating the App with Ragas

Now we can finally do our evaluation!

We'll start by running the queries we generated usign SDG above through our application to get context and responses.

In [28]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [29]:
dataset.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,synthesizer_name
0,Wht did we lern about LLMs in 2024?,[This is Things we learned about LLMs in 2024 ...,[Code may be the best application The ethics o...,"In 2024, we learned several key things about L...",The most surprising thing we’ve learned about ...,single_hop_specifc_query_synthesizer
1,How do LLMs handle the complexity of Chinese g...,[So training an LLM still isn’t something a ho...,[Based Development As a computer scientist and...,LLMs handle the complexity of Chinese grammar ...,The grammar rules of programming languages lik...,single_hop_specifc_query_synthesizer
2,What were the key developments in AI in 2023?,[Law is not ethics. Is it OK to train models o...,[Simon Willison’s Weblog Subscribe Stuff we fi...,The key developments in AI in 2023 included:\n...,2023 was the breakthrough year for Large Langu...,single_hop_specifc_query_synthesizer
3,What plausible impact AI models have on jobs?,[Law is not ethics. Is it OK to train models o...,[easy to follow. The rest of the document incl...,The plausible impact of AI models on jobs incl...,People have certainly lost work to AI models—a...,single_hop_specifc_query_synthesizer
4,How have advancements in model efficiency and ...,[Law is not ethics. Is it OK to train models o...,[<1-hop>\n\nVoice and live camera mode are sci...,Advancements in model efficiency and pricing h...,Advancements in model efficiency and pricing h...,multi_hop_abstract_query_synthesizer
5,How has the universal access to AI models evol...,[Law is not ethics. Is it OK to train models o...,[<1-hop>\n\nVoice and live camera mode are sci...,The evolution of universal access to AI models...,Universal access to the best AI models was bri...,multi_hop_abstract_query_synthesizer
6,How have advancements in model efficiency and ...,[Law is not ethics. Is it OK to train models o...,[<1-hop>\n\nVoice and live camera mode are sci...,Advancements in model efficiency and pricing h...,Advancements in model efficiency and pricing h...,multi_hop_abstract_query_synthesizer
7,How has universal access to AI models and API ...,[For a few short months this year all three of...,[<1-hop>\n\nVoice and live camera mode are sci...,"In recent years, universal access to AI models...",Universal access to AI models experienced a br...,multi_hop_abstract_query_synthesizer
8,What were the significant advancements in Larg...,"[Since then, almost every major LLM (and most ...",[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,"In 2024, significant advancements in Large Lan...","In 2024, significant advancements in Large Lan...",multi_hop_specific_query_synthesizer
9,How has the development of Claude 3 and the ri...,[The rise of inference-scaling “reasoning” mod...,[<1-hop>\n\nI’m beginning to see the most popu...,The development of Claude 3 and the rise of in...,The development of Claude 3 and the rise of in...,multi_hop_specific_query_synthesizer


Then we can convert that table into a `EvaluationDataset` which will make the process of evaluation smoother.

In [30]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

We'll need to select a judge model - in this case we're using the same model that was used to generate our Synthetic Data.

In [31]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

Next up - we simply evaluate on our desired metrics!

In [32]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[13]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-1a7uFAatkr7hjd3rqKa2Vpxy on tokens per min (TPM): Limit 30000, Used 29876, Requested 2622. Please try again in 4.996s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[1]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[5]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[7]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[11]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[16]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[17]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[19]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[22]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[24]: TimeoutError()
ERROR:

{'context_recall': 0.6778, 'faithfulness': 0.3913, 'factual_correctness': 0.4200, 'answer_relevancy': 0.9567, 'context_entity_recall': 0.3786, 'noise_sensitivity_relevant': 0.2727}

## Making Adjustments and Re-Evaluating

Now that we've got our baseline - let's make a change and see how the model improves or doesn't improve!

> NOTE: This will be using Cohere's Rerank model - please be sure to sign-up [here](https://cohere.com/rerank) for an API key!


API Key : https://dashboard.cohere.com/api-keys

In [33]:
os.environ["COHERE_API_KEY"] = getpass("Please enter your Cohere API key!")

Please enter your Cohere API key!··········


In [34]:
!pip install -qU cohere langchain_cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.9/252.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.9 MB/s eta 0:00:00


In [35]:
retriever = vector_store.as_retriever(search_kwargs={"k": 20})

In [36]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
  compressor = CohereRerank(model="rerank-english-v3.0")
  compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever, search_kwargs={"k": 5}
  )
  retrieved_docs = compression_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

In [37]:
class State(TypedDict):
  question: str
  context: List[Document]
  response: str

graph_builder = StateGraph(State).add_sequence([retrieve_adjusted, generate])
graph_builder.add_edge(START, "retrieve_adjusted")
graph = graph_builder.compile()

In [38]:
response = graph.invoke({"question" : "How are LLM agents useful?"})
response["response"]

"LLM agents are perceived as potentially useful in various applications, particularly in coding, where they have shown a high level of capability. They can assist in writing code, which is less complex than natural language grammar, making them effective tools in that domain. Additionally, there is excitement surrounding the concept of AI agents that can act on users' behalf, although there is skepticism about their reliability due to concerns over gullibility and the inability to distinguish truth from fiction. Critics emphasize the importance of discussing the ethical implications and challenges associated with LLMs, urging caution and responsible use of these technologies."

In [39]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [40]:
result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[22]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-1a7uFAatkr7hjd3rqKa2Vpxy on tokens per min (TPM): Limit 30000, Used 29400, Requested 1837. Please try again in 2.474s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[2]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-1a7uFAatkr7hjd3rqKa2Vpxy on tokens per min (TPM): Limit 30000, Used 29795, Requested 1587. Please try again in 2.764s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[1]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[5]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[7]: 

{'context_recall': 0.6800, 'faithfulness': 0.4783, 'factual_correctness': 0.4450, 'answer_relevancy': 0.9580, 'context_entity_recall': 0.4098, 'noise_sensitivity_relevant': 0.1667}